<a href="https://colab.research.google.com/github/Shadow977/BERT-training-language-models-on-tpus-from-scratch.ipynb/blob/master/training_language_models_on_tpus_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### In this kernel, I will show you how to train language models, such as BERT, from scratch on TPUs!

#### you can find the bert-base data sets here:(Rest are in repository)

- https://www.kaggle.com/abhishek/bert-base-uncased

Update your tokenizers to add support for hugging face tokenizer

In [ ]:
! pip install -U tokenizers==0.4.2

Since huggingface library doesnt support TF2.0,Downgrade tensorflow to 1.15

In [ ]:
!pip install tensorflow==1.15

In [ ]:
import tokenizers

In [ ]:
bwpt = tokenizers.BertWordPieceTokenizer(
    vocab_file=None,
    add_special_tokens=True,
    unk_token='[UNK]',
    sep_token='[SEP]',
    cls_token='[CLS]',
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=True,
    lowercase=True,
    wordpieces_prefix='##'
)

Import BERT source and BERT base uncased files

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import shutil
shutil.copy('/content/gdrive/My Drive/Collab Data/BERT_base_uncased.zip','/content/')
shutil.copy('/content/gdrive/My Drive/Collab Data/BERTsrc.zip','/content/')

!unzip BERT_base_uncased.zip
!unzip '/content/BERTsrc.zip'

Import the dataset

In [ ]:
!wget https://github.com/Shadow977/BERT-training-language-models-on-tpus-from-scratch.ipynb/blob/master/hi_dedup_1000.txt

Train tokenizer on dataset

In [ ]:
bwpt.train(
    files=["/content/hi_dedup_1000.txt"],
    vocab_size=30000,
    min_frequency=3,
    limit_alphabet=1000,
    special_tokens=['[PAD]', '[UNK]', '[CLS]', '[MASK]', '[SEP]']
)

Save the weights

In [ ]:
bwpt.save("/content/", "hindi")

In [ ]:
cd /content/

In [ ]:
!ls

In [ ]:
!python create_pretraining_data.py \
    --input_file=hi_dedup_1000.txt \
    --output_file=tf_examples.tfrecord \
    --vocab_file=hindi-vocab.txt \
    --do_lower_case=True \
    --max_seq_length=128 \
    --max_predictions_per_seq=20 \
    --masked_lm_prob=0.15 \
    --random_seed=42 \
    --dupe_factor=5

In [ ]:
from google.colab import auth
auth.authenticate_user()

To access the storage you need to authenticate and get storage bucket, for that you'll need colab free trail or colab pro.
you can get the bucket name [here](https://console.cloud.google.com/storage/).

`where the SERVICE_ACCOUNT is service-[PROJECT_NUMBER]@cloud-tpu.iam.gserviceaccount.com`

The PROJECT_NUMBER for Colab is contained in the error message of the response when TPU is not authenticated.

So, first run your TPU service without TPU auth, waiting for the error message from the response, then Enable TPU auth.

In [ ]:
!gsutil ls gs://[BUCKET_NAME]
gsutil acl ch -u [SERVICE_ACCOUNT]:READER gs://[BUCKET_NAME]
gsutil acl ch -u [SERVICE_ACCOUNT]:WRITER gs://[BUCKET_NAME]

In [ ]:
!python run_pretraining.py \
    --input_file=gs://tf-lang-model/*.tfrecord \
    --output_dir=gs://tf-lang-model/model/ \
    --do_train=True \
    --do_eval=True \
    --bert_config_file=/content/config.json \
    --train_batch_size=32 \
    --max_seq_length=128 \
    --max_predictions_per_seq=20 \
    --num_train_steps=20 \
    --num_warmup_steps=10 \
    --learning_rate=2e-5 \
    --use_tpu=True \
    --tpu_name=$TPU_NAME